

Hãy phân tích các cases có thể gây lỗi khi người dùng chọn số lượng câu hỏi:

1. **Case 1: Số câu hỏi yêu cầu > Tổng số câu có sẵn**
```python
# Ví dụ: Chủ đề có 5 câu (3 TL, 2 TN), user yêu cầu 10 câu
total_available_questions = len(mc_questions) + len(essay_questions)  # = 5
num_questions = min(num_questions, total_available_questions)  # = min(10, 5) = 5
# ✅ Đã xử lý OK bằng hàm min()
```

2. **Case 2: Chỉ có câu tự luận, không có trắc nghiệm**
```python
# Ví dụ: Chủ đề có 3 TL, 0 TN, user yêu cầu 3 câu
desired_essay = int(3 * 0.7) + 1  # = 3
actual_essay = min(desired_essay, len(essay_questions))  # = min(3, 3) = 3
actual_mc = min(num_questions - actual_essay, len(mc_questions))  # = min(0, 0) = 0
# ✅ Đã xử lý OK
```

3. **Case 3: Chỉ có câu trắc nghiệm, không có tự luận**
```python
# Ví dụ: Chủ đề có 0 TL, 5 TN, user yêu cầu 3 câu
desired_essay = int(3 * 0.7) + 1  # = 3
actual_essay = min(desired_essay, len(essay_questions))  # = min(3, 0) = 0
actual_mc = min(num_questions - actual_essay, len(mc_questions))  # = min(3, 5) = 3
# ✅ Đã xử lý OK
```

4. **Case 4: Số câu yêu cầu < Số câu tự luận mong muốn**
```python
# Ví dụ: Chủ đề có 5 TL, 3 TN, user yêu cầu 2 câu
desired_essay = int(2 * 0.7) + 1  # = 2
actual_essay = min(desired_essay, len(essay_questions))  # = min(2, 5) = 2
actual_mc = min(num_questions - actual_essay, len(mc_questions))  # = min(0, 3) = 0
# ⚠️ Có thể cân nhắc điều chỉnh công thức tính desired_essay
```

5. **Case 5: Không có câu hỏi nào trong chủ đề**
```python
if total_available_questions == 0:
    raise ValueError("Không có câu hỏi nào cho chủ đề đã chọn.")
# ✅ Đã xử lý OK bằng raise Exception
```

6. **Case 6: Người dùng yêu cầu số câu hỏi = 0 hoặc âm**
```python
# ⚠️ Chưa xử lý case này
# Nên thêm validation:
if num_questions <= 0:
    raise ValueError("Số lượng câu hỏi phải lớn hơn 0")
```

7. **Case 7: Tổng số câu được chọn < Số câu yêu cầu**
```python
selected_questions = selected_essay + selected_mc
# Có thể xảy ra khi actual_essay + actual_mc < num_questions
# ⚠️ Nên thêm log cảnh báo
if len(selected_questions) < num_questions:
    logger.warning(f"Chỉ có thể lấy {len(selected_questions)} câu thay vì {num_questions} câu theo yêu cầu")
```

Đề xuất sửa đổi:
```python
def initialize_quiz(self, user_name, selected_topics, num_questions):
    # Validate input
    if num_questions <= 0:
        raise ValueError("Số lượng câu hỏi phải lớn hơn 0")

    mc_questions, essay_questions = self.data_loader.get_questions_by_topic(selected_topics)
    total_available_questions = len(mc_questions) + len(essay_questions)

    if total_available_questions == 0:
        raise ValueError("Không có câu hỏi nào cho chủ đề đã chọn.")

    num_questions = min(num_questions, total_available_questions)
    desired_essay = int(num_questions * 0.7) + 1
    actual_essay = min(desired_essay, len(essay_questions))
    actual_mc = min(num_questions - actual_essay, len(mc_questions))

    selected_essay = random.sample(essay_questions, actual_essay) if essay_questions else []
    selected_mc = random.sample(mc_questions, actual_mc) if mc_questions else []

    selected_questions = selected_essay + selected_mc
    
    if len(selected_questions) < num_questions:
        logger.warning(f"Chỉ có thể lấy {len(selected_questions)} câu thay vì {num_questions} câu theo yêu cầu")

    # ... rest of the code ...
```